In [44]:
import random
import os

words = ['Environment', 'Archives', 'Pronounciation', 
         'Hour', 'Wednesday', 'Violence', 'Tomb', 'Suite', 
         'Iron', 'Reciept', 'Chores']
random_words = random.sample(words, 5)
random_string = ' '.join(random_words)
print(random_string)

# Directory path to traverse
directory_path = 'models/'
models = []

for word in random_words:
    for file in os.listdir(directory_path):
        if word in file:
            models.append(file)

model = joblib.load("models/" + models[0])
model.predict()


Suite Pronounciation Archives Iron Wednesday


RandomForestClassifier(random_state=42)

In [41]:
import sounddevice as sd
import librosa

def extract_mfcc(audio_array, sample_rate):
    mfccs = librosa.feature.mfcc(y=audio_array, sr=sample_rate)
    return mfccs.tolist()

freq = 44100
duration = 2
recording = sd.rec(int(duration * freq), samplerate=freq, channels=2, dtype='float32')
sd.wait()  # Wait for the recording to complete
y = recording[:, 0]  # Use only one channel if you have stereo recording
sr = freq

mfcc_features = extract_mfcc(y, sr)
print(mfcc_features)

[[-756.1118774414062, -756.1118774414062, -756.1118774414062, -756.1118774414062, -756.1118774414062, -756.1118774414062, -756.1118774414062, -754.5108642578125, -709.3975830078125, -635.2264404296875, -566.9818725585938, -512.4885864257812, -484.9091796875, -457.3285217285156, -420.2255554199219, -388.3809814453125, -365.34814453125, -342.4414367675781, -320.2181091308594, -306.53662109375, -296.38946533203125, -282.3834228515625, -273.56500244140625, -252.5856170654297, -239.3944854736328, -243.42381286621094, -247.49969482421875, -244.05636596679688, -240.16322326660156, -242.2339630126953, -243.2291717529297, -244.32601928710938, -240.903564453125, -239.3055419921875, -234.07801818847656, -239.0821533203125, -240.5382843017578, -238.6155548095703, -239.2743682861328, -241.13034057617188, -236.8671112060547, -242.29718017578125, -246.44949340820312, -238.65744018554688, -244.6901397705078, -245.62188720703125, -242.02999877929688, -238.87545776367188, -241.8879852294922, -240.985778

In [45]:
import joblib
import ast

def extract_mfcc(audio_array, sample_rate):
    mfccs = librosa.feature.mfcc(y=audio_array, sr=sample_rate)
    return mfccs.tolist()

freq = 44100
duration = 2
users = []

for i in range(len(random_words)):
    print(f'Please pronounce this word - {random_words[i]}')
    recording = sd.rec(int(duration * freq), samplerate=freq, channels=2, dtype='float32')
    sd.wait()  # Wait for the recording to complete
    y = recording[:, 0]  # Use only one channel if you have stereo recording
    sr = freq
    mfcc_features = extract_mfcc(y, sr)
    mfcc_features = mfcc_features.apply(ast.literal_eval)
    mfcc_features = mfcc_features.apply(lambda x: [i])
    loaded_model = joblib.load("models/" + models[i])
    user = loaded_model.predict(mfcc_features)
    users.append(user)

users

Please pronounce this word - Suite


ValueError: X has 173 features, but RandomForestClassifier is expecting 1740 features as input.